In [1]:
import pandas as pd

from code.parser.parse import Parser
import matplotlib.pyplot as plt
from code.wiener_kolmogorov.wiener_kolmogorov import WienerKolmogorov as WK
from code.detector.detector import  Detector
import numpy as np
import datetime

pre1 = Parser.read('../../runs/1/pre.txt')
abs1 = Parser.read('../../runs/1/abs.txt')
pre2 = Parser.read('../../runs/2/pre.txt')
abs2 = Parser.read('../../runs/2/abs.txt')

In [2]:
pre1 = pd.concat([pre1, abs1], ignore_index=True)
abs1 = pd.concat([abs1, abs1], ignore_index=True)
pre2 = pd.concat([pre2, abs2], ignore_index=True)
abs2 = pd.concat([abs2, abs2], ignore_index=True)

In [3]:
pre1 = pre1.rename(columns={"time": "ds"})
pre2 = pre2.rename(columns={"time": "ds"})

In [4]:
pre1 = pd.concat([pre1, pre2], ignore_index=True)

In [5]:
pre1.ds = [(datetime.datetime.now() + datetime.timedelta(seconds=i)) for i in range(0,  400, 1)]
pre2.ds = [(datetime.datetime.now() + datetime.timedelta(seconds=i)) for i in range(0,  200, 1)]

In [6]:
from autosklearn.classification import AutoSklearnClassifier

In [7]:
pre1.rssi.to_numpy()

array([-29., -27., -26., -27., -26., -26., -26., -27., -27., -27., -27.,
       -27., -32., -35., -34., -38., -28., -38., -27., -38., -38., -34.,
       -38., -38., -38., -35., -27., -38., -38., -38., -27., -38., -38.,
       -35., -38., -38., -38., -35., -38., -32., -38., -35., -27., -38.,
       -38., -38., -38., -38., -35., -38., -20., -20., -19., -20., -21.,
       -19., -21., -18., -22., -20., -20., -20., -19., -22., -19., -18.,
       -22., -23., -18., -19., -21., -21., -20., -21., -18., -21., -22.,
       -22., -20., -21., -24., -20., -23., -20., -20., -22., -24., -19.,
       -20., -17., -19., -18., -20., -20., -19., -21., -22., -20., -20.,
       -19., -20., -20., -19., -20., -21., -19., -21., -18., -22., -20.,
       -20., -20., -19., -22., -19., -18., -22., -23., -18., -19., -21.,
       -21., -20., -21., -18., -21., -22., -22., -20., -21., -24., -20.,
       -23., -20., -20., -22., -24., -19., -20., -17., -19., -18., -20.,
       -20., -19., -21., -22., -20., -20., -19., -1

In [8]:
ac = AutoSklearnClassifier(memory_limit=5000, time_left_for_this_task=120)
ac.fit(pre1.rssi.to_numpy()[:,np.newaxis], pre1.y.to_numpy())

AttributeError: 'DataFrame' object has no attribute 'y'

In [9]:
ac.leaderboard()

AttributeError: 'NoneType' object has no attribute 'runhistory_'

In [ ]:
predict = ac.predict(pre2.rssi.to_numpy()[:,np.newaxis])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(pre2.y.to_numpy(), predict)

In [ ]:
def filter_out(pres, abss):
    filters = []
    for abs in abss:
        filter = WK()
        filter.fit(abs["rssi"].to_numpy())
        filters.append(filter)
    outs = []
    print(filters)
    for i, pre in enumerate(pres):
        out = filters[i].transform(pre["rssi"].to_numpy())
        outs.append(out)
    return outs, filters

In [ ]:
outs, _ = filter_out([pre1], [abs1])

In [ ]:
from scipy.signal import medfilt, order_filter

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(pre1.rssi.to_numpy()[:,np.newaxis], pre1.y)
predict = forest.predict(pre2.rssi.to_numpy()[:,np.newaxis])
accuracy_score(pre2.y.to_numpy(), medfilt(predict, kernel_size=13))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
forest = AdaBoostClassifier()
forest.fit(pre1.rssi.to_numpy()[:,np.newaxis], pre1.y)
predict = forest.predict(pre2.rssi.to_numpy()[:,np.newaxis])
accuracy_score(pre2.y.to_numpy(), medfilt(predict, kernel_size=13))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
forest = GradientBoostingClassifier(learning_rate=0.00001)
forest.fit(pre1.rssi.to_numpy()[:,np.newaxis], pre1.y)
predict = forest.predict(pre2.rssi.to_numpy()[:,np.newaxis])
accuracy_score(pre2.y.to_numpy(), medfilt(predict, kernel_size=13))

In [ ]:
forest.n_estimators

In [ ]:
# axs.append(fig.add_subplot(n, 1, k))
#         axs[k - 1].plot(pre.rssi.to_numpy(), label=legend[0], linestyle="solid")
#         axs[k-1].set_xticks(ticks)
#         axs[k-1].set_xlabel("Время, с")
#         axs[k-1].set_ylabel("RSSI, dBm")
#         axs[k - 1].plot(out, label=legend[1], linestyle="dashed")
#         # axs[k - 1].legend()
#         axs[k - 1].set(title=titles[k-1])
plt.rcParams.update({'font.size': 25})
plt.figure(figsize=(20, 6))
plt.plot(pre1.rssi.to_numpy(), label="RSSI до обработки", linestyle=":", marker=".")
plt.plot(outs[0], label="RSSI после обработки", linestyle="--", marker="s")
plt.title("Замеры RSSI в жилой комнате")
plt.legend(loc="lower right")
plt.savefig("shibaev_1.png")

In [ ]:
plt.rcParams.update({'font.size': 25})
plt.figure(figsize=(20, 6))
plt.plot(pre1.rssi.to_numpy(), label="RSSI до обработки", linestyle=":", marker=".")
plt.plot(outs[0], label="RSSI после обработки", linestyle="--", marker="s")
plt.title("Замеры RSSI в офисе")
plt.legend(loc="lower right")
plt.savefig("shibaev_2.png")